In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import geopandas as gpd
from geopy.distance import distance,geodesic
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

In [26]:
# please download data at https://www.dropbox.com/s/uquijy335rg0kjn/date-hour-soo-dest-2022.csv.gz?dl=0
data = pd.read_csv('date-hour-soo-dest-2022.csv.gz',compression='gzip',header=None)
data.columns = ['Date', 'Hour', 'Origin', 'Destination', 'Number']

In [27]:
data['Month'] = data['Date'].apply(lambda x: x.split('-')[1])
data = data.loc[data['Month'].isin(['09','10'])]
data['OD'] = data['Origin'] + ' - ' + data['Destination']

In [28]:
data = pd.pivot_table(data,index=['Date','Hour'],columns=['OD'],fill_value=0).reset_index()
data.columns = [i[1] if i[0]=='Number' else i[0] for i in data.columns]

In [29]:

data = data.merge(pd.DataFrame({'Date':sorted(data['Date'].unique().tolist()*24),
                         'Hour':list(range(0,24))*len(data['Date'].unique())}),
                  on=['Date','Hour'],how='outer').\
fillna(0).sort_values(by=['Date','Hour'])

data[data.columns[2:]] = data[data.columns[2:]].astype('float16')

In [30]:
od_flow = data.melt(id_vars=['Date','Hour']).fillna(0)
od_flow['o'] = od_flow['variable'].apply(lambda x:x.split(' - ')[0])
od_flow['d'] = od_flow['variable'].apply(lambda x:x.split(' - ')[1])


outgoing_flow = od_flow.groupby(['Date','Hour','o']).agg({'value':sum}).reset_index()
outgoing_flow.rename(columns={'o':'station','value':'outgoing_flow'},inplace=True)
incoming_flow = od_flow.groupby(['Date','Hour','d']).agg({'value':sum}).reset_index()
incoming_flow.rename(columns={'d':'station','value':'incoming_flow'},inplace=True)

flow = incoming_flow.merge(outgoing_flow,on=['Date','Hour','station'])
od = flow.pivot_table(values=['incoming_flow','outgoing_flow'],index=['Date','Hour'],
                columns = 'station')
col = od.columns
od.columns = [i[0]+'-'+i[1] for i in col]
od.to_csv('inout.csv')

# different methods, all tested on 09-26 to 10-02

In [31]:
od = od.astype('float128')
od.head()

incoming_flow-12TH  incoming_flow-16TH  incoming_flow-19TH  \
Date       Hour                                                               
2022-09-01 0                   21.0                38.0                17.0   
           1                    1.0                 3.0                 1.0   
           2                    0.0                 0.0                 0.0   
           3                    0.0                 0.0                 0.0   
           4                    0.0                 0.0                 0.0   

                 incoming_flow-24TH  incoming_flow-ANTC  incoming_flow-ASHB  \
Date       Hour                                                               
2022-09-01 0                   48.0                29.0                38.0   
           1                    2.0                13.0                 1.0   
           2                    0.0                 5.0                 0.0   
           3                    0.0                 0.0                 0.0   
           4                    1.0                 3.0                 0.0   

                 incoming_flow-BALB  incoming_flow-BAYF  incoming_flow-BERY  \
Date       Hour                                                               
2022-09-01 0                   38.0                21.0               101.0   
           1                    8.0                 2.0                 3.0   
           2                    0.0                 0.0                 1.0   
           3                    0.0                 0.0                 0.0   
           4                    0.0                 0.0                 0.0   

                 incoming_flow-CAST  ...  outgoing_flow-SANL  \
Date       Hour                      ...                       
2022-09-01 0                   34.0  ...                10.0   
           1                    2.0  ...                 0.0   
           2                    0.0  ...                 0.0   
           3                    0.0  ...                 0.0   
           4                    0.0  ...                 0.0   

                 outgoing_flow-SBRN  outgoing_flow-SFIA  outgoing_flow-SHAY  \
Date       Hour                                                               
2022-09-01 0                    4.0                89.0                 2.0   
           1                    1.0                10.0                 1.0   
           2                    0.0                 2.0                 0.0   
           3                    0.0                 0.0                 0.0   
           4                    0.0                 1.0                 0.0   

                 outgoing_flow-SSAN  outgoing_flow-UCTY  outgoing_flow-WARM  \
Date       Hour                                                               
2022-09-01 0                    4.0                 4.0                12.0   
           1                    1.0                 1.0                 1.0   
           2                    0.0                 0.0                 0.0   
           3                    0.0                 0.0                 0.0   
           4                    0.0                 4.0                 2.0   

                 outgoing_flow-WCRK  outgoing_flow-WDUB  outgoing_flow-WOAK  
Date       Hour                                                              
2022-09-01 0                    2.0                 4.0                 7.0  
           1                    3.0                 0.0                 5.0  
           2                    0.0                 0.0                 0.0  
           3                    0.0                 0.0                 0.0  
           4                    0.0                 0.0                 0.0  

[5 rows x 100 columns]

In [32]:
### same as 1 hours before
print(r2_score(od.iloc[-24*7-1:-1,],od.iloc[-24*7:,],multioutput='variance_weighted'))
print(r2_score(od.iloc[-24*7-1:-1,].values,od.iloc[-24*7:,].values))


0.6606124503714431
0.6201865778817927


In [33]:
print(mean_absolute_error(od.iloc[-24*7-1:-1,].values,od.iloc[-24*7:,].values))

47.4725


In [35]:
### same as a week ago
print(r2_score(od.iloc[-48*7:-24*7,].values,od.iloc[-24*7:,].values))
print(mean_absolute_error(od.iloc[-48*7:-24*7,].values,od.iloc[-24*7:,].values))
print(mean_squared_error(od.iloc[-48*7:-24*7,].values,od.iloc[-24*7:,].values,squared=False))

0.9088078785342216
19.212797619047619046
34.819948936320005026


In [36]:
od = incoming_flow.merge(outgoing_flow,on=['Date','Hour','station'])
od = od.sort_values(by=['Date','Hour','station'])
od['DOW'] = pd.to_datetime(od['Date'])
od['DOW'] = od.DOW.dt.dayofweek
od = pd.concat([od.drop(['DOW'],axis=1),
                     pd.get_dummies(od['DOW'],prefix='dow',prefix_sep='-')],
                   axis=1)
od = pd.concat([od,
                     pd.get_dummies(od['Hour'],prefix='hour',prefix_sep='-')],
                   axis=1)

In [37]:
### lag linear regression, many to one
od = od.sort_values(by=['station','Date','Hour'])
no_lag = 24*7

for lag in range(1,no_lag+1):
    temp = od[['station','incoming_flow','outgoing_flow']].shift(lag)
    temp.columns = ['station'+'-'+str(lag),'incoming_flow'+'-'+str(lag),'outgoing_flow'+'-'+str(lag)]
    od = pd.concat([od,temp],axis=1)
    
od = od.sort_values(by=['Date','Hour','station'])
od = od.dropna()
od = od.loc[od['station']==od['station'+'-'+str(lag)]]

x = od[[col for col in od.columns if '_flow-' in col]]
y = od[['incoming_flow','outgoing_flow']]

x_train = x.iloc[:-24*50*7,:].values
y_train = y.iloc[:-24*50*7,].values

model = LinearRegression(fit_intercept=False).fit(x_train, y_train)

print('out of sample R2')
x_test = x.iloc[-24*50*7:,:].values
y_test = y.iloc[-24*50*7:].values
y_pred = model.predict(x_test)

print(r2_score(y_pred,y_test))
print(r2_score(y_pred.reshape(24*7*50,2),y_test.reshape(24*7*50,2),multioutput='variance_weighted'))
print(mean_absolute_error(y_pred,y_test))
print(mean_squared_error(y_pred,y_test,squared=False))


out of sample R2
0.9658714587933703
0.9656071130380675
22.377527
41.379025


## adding  surrounding flows

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.__version__

'1.11.0'

In [40]:
import torch_geometric
from torch_geometric.utils import to_dense_adj

od = incoming_flow.merge(outgoing_flow,on=['Date','Hour','station'])
od = od.sort_values(by=['Date','Hour','station'])
od['DOW'] = pd.to_datetime(od['Date'])
od['DOW'] = od.DOW.dt.dayofweek
od = pd.concat([od.drop(['DOW'],axis=1),
                     pd.get_dummies(od['DOW'],prefix='dow',prefix_sep='-')],
                   axis=1)
od = pd.concat([od,
                     pd.get_dummies(od['Hour'],prefix='hour',prefix_sep='-')],
                   axis=1)


In [41]:
od.iloc[:,5:] = od.iloc[:,5:].astype('float128')

In [42]:
# create adj matrix
stations = od['station'].unique()
stations_index = dict(zip(stations,range(50)))

In [43]:
yellow = ['ANTC','PCTR','PITT','NCON','CONC','PHIL','WCRK','LAFY',
        'ORIN','ROCK','MCAR','19TH','12TH','WOAK','EMBR',
        'MONT','POWL','CIVC','16TH','24TH','GLEN','BALB',
        'DALY','COLM','SSAN','SBRN','MLBR','SFIA']
orange = ['RICH','DELN','PLZA','NBRK','DBRK','ASHB','MCAR',
          '19TH','12TH','LAKE','FTVL','COLS','SANL','BAYF',
          'HAYW','SHAY','UCTY','FRMT','WARM','MLPT','BERY']
red = ['RICH','DELN','PLZA','NBRK','DBRK','ASHB','MCAR',
          '19TH','12TH','WOAK','EMBR',
        'MONT','POWL','CIVC','16TH','24TH','GLEN','BALB',
        'DALY','COLM','SSAN','SBRN','MLBR','SFIA']
blue = ['DUBL','WDUB','CAST','BAYF','SANL','COLS','FTVL',
        'LAKE','WOAK','EMBR','MONT','POWL','CIVC','16TH',
        '24TH','GLEN','BALB','DALY']
green = ['BERY','MLPT','WARM','FRMT','UCTY','SHAY','HAYW',
         'BAYF','SANL','COLS','FTVL',
        'LAKE','WOAK','EMBR','MONT','POWL','CIVC','16TH',
        '24TH','GLEN','BALB','DALY']
grey = ['COLS','OAKL']

In [44]:
adj = np.zeros([50,50])
for route in [yellow,orange,red,blue,green,grey]:
    i = 0
    for station in route:
        if i+1 < len(route):
            pair1 = stations_index[route[i]]
            pair2 = stations_index[route[i+1]]
            adj[pair1,pair2] = adj[pair1,pair2]+1
            adj[pair2,pair1] = adj[pair2,pair1]+1
            i += 1

In [45]:
connection = {}
for route in [yellow,orange,red,blue,green,grey]:
    i = 0
    for station in route:
        if i+1 < len(route):
            key = route[i]
            value = route[i+1]
            connection[key] = connection.get(key,[])+[value]
            connection[value] = connection.get(value,[])+[key]
            i += 1
for key in connection.keys():
    connection[key] = list(set(connection[key]))

In [46]:
def get_nearby_flow(x,od,connection):
    date = x['Date']
    hour = x['Hour']
    source = x['station']
    
    temp = od.loc[(od['Date']==date)&\
                                    (od['Hour']==hour)]
    nearby_flows = temp.loc[(temp['station'].isin(connection[source]))]\
                                    [['incoming_flow','outgoing_flow']].sum().values.tolist()

    
    return nearby_flows
    

In [21]:
nearby_flow_list = Parallel(n_jobs=8)(delayed(get_nearby_flow)(od.iloc[i],od,connection) for i in range(len(od)))
od[['nearby_incoming','nearby_outgoing']] = nearby_flow_list

In [22]:
od['nearby_incoming'] = (od['nearby_incoming']+0.1)/(od['incoming_flow']+0.1)
od['nearby_outgoing'] = (od['nearby_outgoing']+0.1)/(od['incoming_flow']+0.1)

In [23]:
od = od.sort_values(by=['station','Date','Hour'])
no_lag = 24*7
for lag in range(1,no_lag+1):
    temp = od[['station','incoming_flow','outgoing_flow','nearby_incoming','nearby_outgoing']].shift(lag)
    temp.columns = ['station'+'-'+str(lag),
                    'incoming_flow'+'-'+str(lag),'outgoing_flow'+'-'+str(lag),
                    'nearby_incoming'+'-'+str(lag),'nearby_outgoing'+'-'+str(lag)]
    od = pd.concat([od,temp],axis=1)
    od['incoming_flow'+'-'+str(lag)] = (od['incoming_flow'+'-'+str(lag)]+0.1)/(od['incoming_flow']+0.1)
    od['outgoing_flow'+'-'+str(lag)] = (od['outgoing_flow'+'-'+str(lag)]+0.1)/(od['incoming_flow']+0.1)
    od['nearby_incoming'+'-'+str(lag)] = (od['nearby_incoming'+'-'+str(lag)]+0.1)/(od['incoming_flow']+0.1)
    od['nearby_outgoing'+'-'+str(lag)] = (od['nearby_outgoing'+'-'+str(lag)]+0.1)/(od['incoming_flow']+0.1)
    
od = od.sort_values(by=['Date','Hour','station'])
od = od.dropna()
od = od.loc[od['station']==od['station'+'-'+str(lag)]]
od = od.drop(columns=[i for i in od.columns if 'station-' in i])

In [25]:
od.to_csv('OD_168_neighbor.csv',index=False)

In [47]:
od = pd.read_csv('OD_168_neighbor.csv')

x = od[od.columns[5:]]
y = od[['incoming_flow','outgoing_flow']]

x_train = x.iloc[:-24*50*7,:].values
y_train = y.iloc[:-24*50*7,].values

model = LinearRegression(fit_intercept=False).fit(x_train, y_train)

print('out of sample R2')
x_test = x.iloc[-24*50*7:,:].values
y_test = y.iloc[-24*50*7:].values
y_pred = model.predict(x_test)

print(r2_score(y_pred,y_test))
print(r2_score(y_pred.reshape(24*7*50,2),y_test.reshape(24*7*50,2),multioutput='variance_weighted'))
print(mean_absolute_error(y_pred,y_test))
print(mean_squared_error(y_pred,y_test,squared=False))

out of sample R2
-3.0058086291145685
-2.9952399813297865
95.58156300187923
203.12540831490435


In [48]:
# regularization, LASSO
from sklearn.linear_model import MultiTaskLasso
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

parameters = {'alpha':np.arange(0.01,1,0.1), 'fit_intercept':[True]}
lasso = MultiTaskLasso(max_iter=1000)
lassocv = GridSearchCV(lasso, parameters,n_jobs=-1,scoring='neg_mean_absolute_error',verbose=3)
lassocv.fit(x_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1950: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 569702044.6305478, tolerance: 140305.3694035185
  cd_fast.enet_coordinate_descent_multi_task(
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1950: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 599579368.1648469, tolerance: 147789.82768619794
  cd_fast.enet_coordinate_descent_multi_task(
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1950: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 690074818.4409485, tolerance: 169735.87804351852
  cd_fast.enet_coordinate_descent_multi_task(
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordi

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1950: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 677874672.6733183, tolerance: 168354.20647136576
  cd_fast.enet_coordinate_descent_multi_task(
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1950: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 581243923.2932562, tolerance: 140305.3694035185
  cd_fast.enet_coordinate_descent_multi_task(
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1950: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 713430306.4163955, tolerance: 174850.09336642938
  cd_fast.enet_coordinate_descent_multi_task(
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordi

GridSearchCV(estimator=MultiTaskLasso(), n_jobs=-1,
             param_grid={'alpha': array([0.01, 0.11, 0.21, 0.31, 0.41, 0.51, 0.61, 0.71, 0.81, 0.91]),
                         'fit_intercept': [True]},
             scoring='neg_mean_absolute_error', verbose=3)

In [49]:
y_pred = lassocv.best_estimator_.predict(x_test)
print(r2_score(y_pred,y_test))
print(mean_absolute_error(y_pred,y_test))
print(mean_squared_error(y_pred,y_test,squared=False))

-3.508738209715691
95.4770053297275
202.88609299410282


In [50]:
# PCA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

# set the tolerance to a large value to make the example faster
regressor = LinearRegression()
pipe = Pipeline(steps=[("pca", pca), ("regressor", regressor)])

# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {
    "pca__n_components": np.arange(2,705,10),
}
search = GridSearchCV(pipe, param_grid,n_jobs=-1,scoring='neg_mean_absolute_error',verbose=3)
search.fit(x_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Fitting 5 folds for each of 71 candidates, totalling 355 fits
Best parameter (CV score=-101.642):
{'pca__n_components': 702}


In [51]:
y_pred = search.best_estimator_.predict(x_test)
print(r2_score(y_pred,y_test))
print(mean_absolute_error(y_pred,y_test))
print(mean_squared_error(y_pred,y_test,squared=False))

-2.286028101268884
99.04821761720204
212.32559604209655


In [5]:
class VNN(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(VNN, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Linear(input_size, n_feature),
        nn.Sigmoid(),
        nn.Linear(n_feature,n_feature),
        nn.Sigmoid(),
        nn.Linear(n_feature,n_feature),
        nn.Sigmoid(),
        nn.Linear(n_feature,n_feature),
        nn.Sigmoid(),
        nn.Linear(n_feature,n_feature),
        nn.Sigmoid(),
        nn.Linear(n_feature,output_size),
        )
 
    def forward(self, x, verbose=False):
        x = self.layer1(x)        
        return x

def get_loss_and_metrics(model, batch, criterion, device):
  # Implement forward pass and loss calculation for one batch.
  # Remember to move the batch to device.
  # 
  # Return a tuple:
  # - loss for the batch (Tensor)
  # - number of correctly classified examples in the batch (Tensor)
    data, target = batch[0], batch[1]
    data = torch.tensor(data)
    target = torch.tensor(target)
    
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    pred = model(data.float())
    loss = criterion(pred, target)
    
    
    return (pred,target,loss)
    
def step(loss, optimizer):
  # Implement backward pass and update.

  # TODO
    loss = loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [6]:
fts = od.iloc[:-24*50*7,5:].values
scaler = StandardScaler()
scaler.fit(fts)
fts = torch.tensor(scaler.transform(fts))
target = torch.tensor(od.iloc[:-24*50*7,3:5].values)                      
train_dataset = torch.utils.data.TensorDataset(fts,target)
                      
fts_val = torch.tensor(scaler.transform(od.iloc[-24*50*7:,5:].values))                   
target_val = torch.tensor(od.iloc[-24*50*7:,3:5].values)                      
validation_dataset = torch.utils.data.TensorDataset(fts_val, target_val)

In [14]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

N_EPOCHS = 5000
BATCH_SIZE = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=len(validation_dataset),
                                                    num_workers=0)
model = VNN(input_size=od.shape[1]-5,n_feature=64,output_size=2)

criterion = nn.SmoothL1Loss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001) 

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

train_losses = []
validation_losses = []

pbar = tqdm(range(N_EPOCHS))

validation_mae = 99999999
for i in pbar: 
    while validation_mae > 10:
        total_train_loss = 0.0
        total_train_mae = 0.0
        total_train_r2 = 0.0
        model.train()

        for batch in train_dataloader:
            y_pred,y_true,loss = get_loss_and_metrics(model, batch, criterion, device)
            step(loss.float(), optimizer)
            total_train_loss += loss.item()
            size = y_pred.shape[0]
            mae = mean_absolute_error(y_pred.detach().numpy(),y_true.detach().numpy())
            r2 = r2_score(y_pred.view(size,2).detach().numpy(),y_true.view(size,2).detach().numpy())
            total_train_mae += mae
            total_train_r2 += r2

        with torch.no_grad():       
            y_pred,y_true,loss = get_loss_and_metrics(model, batch, criterion, device)
            total_validation_loss += loss.item()
            size = y_pred.shape[0]
            validation_mae = mean_absolute_error(y_pred.view(size,2).detach().numpy(),y_true.view(size,2).detach().numpy())
            validation_r2 = r2_score(y_pred.view(size,2).detach().numpy(),y_true.view(size,2).detach().numpy())


        mean_train_loss = total_train_loss / len(train_dataloader)
        train_mae = total_train_mae / len(train_dataloader)
        train_r2 = total_train_r2 / len(train_dataloader)

        mean_validation_loss = total_validation_loss / len(validation_dataloader)



        pbar.set_description('train_loss:'+ str(round(mean_train_loss,4))+ \
                             ' validation_mae:'+ str(round(validation_mae,4)) +\
                            ' validation R2: '+ str(round(validation_r2,4)))

train_loss:49.9111 validation_mae:9.6309 validation R2: 0.8178: 100%|█| 5000/500


In [20]:
mean_squared_error(y_pred.view(size,2).detach().numpy(),y_true.view(size,2).detach().numpy(),squared=False)

16.750567010181815